# 서울특별시 공공자전거 따릉이 대여이력 연도별 전처리

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
%ls data/

bike_rent_station.csv
bike_rent_station_gu.csv
seoul-bike-2016.csv
seoul-bike-2017.csv
seoul-bike-2018.csv
서울_공공자전거_따릉이_이용현황자료_20161026.xlsx
서울특별시 공공자전거 대여이력 정보_2015년.csv
서울특별시 공공자전거 대여이력 정보_2016년_1.csv
서울특별시 공공자전거 대여이력 정보_2016년_2.csv
서울특별시 공공자전거 대여이력 정보_2016년_3.csv
서울특별시 공공자전거 대여이력 정보_2017년_1.csv
서울특별시 공공자전거 대여이력 정보_2017년_2분기_1.csv
서울특별시 공공자전거 대여이력 정보_2017년_2분기_2.csv
서울특별시 공공자전거 대여이력 정보_2017년_3분기_1.csv
서울특별시 공공자전거 대여이력 정보_2017년_3분기_2.csv
서울특별시 공공자전거 대여이력 정보_2017년_4분기_1.csv
서울특별시 공공자전거 대여이력 정보_2017년_4분기_2.csv
서울특별시 공공자전거 대여이력 정보_2018년_1분기1.csv
서울특별시 공공자전거 대여이력 정보_2018년_2분기_1.csv
서울특별시 공공자전거 대여이력 정보_2018년_2분기_2.csv
서울특별시 공공자전거 대여이력 정보_2018년_2분기_3.csv
서울특별시 공공자전거 대여이력 정보_2018년_2분기_4.csv
서울특별시 공공자전거 대여정보_201807_01.xlsx
서울특별시 공공자전거 대여정보_201807_02.xlsx
서울특별시 공공자전거 대여정보_201808.xlsx
서울특별시 공공자전거 대여정보_201809_1.xlsx
서울특별시 공공자전거 대여정보_201809_2.xlsx
서울특별시 공공자전거 대여정보_201810_02.xlsx
서울특별시 공공자전거 대여정보_201811.xlsx


In [3]:
def preprocessing(df):
    # 컬럼에 있는 '를 제거한다.
    df.columns = df.columns.str.strip("'")
    # 불필요하게 '가 들어가 있다. 스트링 타입으로 되어있는 데이터에서 '을 제거하자.
    df = df.apply(lambda x: x.str.strip("'") if x.dtype == np.dtype('object') else x)
    return df

In [4]:
# 컬럼명이 2017년을 기점으로 달라지기 때문에 최근 컬럼에 맞게 컬럼명을 변경해 준다.
# 우선 최근 컬럼명을 가져온다.
df_for_columns = pd.read_csv("data/서울특별시 공공자전거 대여이력 정보_2018년_2분기_4.csv", encoding="cp949")
df_for_columns.columns = df_for_columns.columns.str.strip("'")
cols = df_for_columns.columns
len(cols)

11

In [5]:
# df = pd.read_excel('data/서울특별시 공공자전거 대여정보_201807_01.xlsx', encoding='cp949')
# df.head()

In [6]:
year = 2018
for root, dirs, files in os.walk("data/"):
    for filename in files:
        if filename.startswith('서울특별시 공공자전거 대여이력 정보_{}년'.format(year)):
            print(filename)
        elif filename.startswith('서울특별시 공공자전거 대여정보_{}'.format(year)):
            print(filename)

서울특별시 공공자전거 대여이력 정보_2018년_1분기1.csv
서울특별시 공공자전거 대여정보_201810_02.xlsx
서울특별시 공공자전거 대여이력 정보_2018년_2분기_1.csv
서울특별시 공공자전거 대여이력 정보_2018년_2분기_3.csv
서울특별시 공공자전거 대여이력 정보_2018년_2분기_2.csv
서울특별시 공공자전거 대여정보_201809_2.xlsx
서울특별시 공공자전거 대여이력 정보_2018년_2분기_4.csv
서울특별시 공공자전거 대여정보_201807_01.xlsx
서울특별시 공공자전거 대여정보_201811.xlsx
서울특별시 공공자전거 대여정보_201808.xlsx
서울특별시 공공자전거 대여정보_201807_02.xlsx
서울특별시 공공자전거 대여정보_201809_1.xlsx


In [7]:
folder_path = "data/"

def concat_data(year):
    df_lists = []
    # 파일별로 데이터를 가져온다.
    # TODO : 해당연도의 데이터만 가져오도록 변경한다.
    for root, dirs, files in os.walk(folder_path):  
        for filename in files:
            df_temp = read_file(filename, year)
            if df_temp.empty != True :
                print(filename, df_temp.shape)
                df_lists.append(df_temp)
    df = pd.concat(df_lists)
    print(df.shape)
    return df        

In [8]:
def read_file(filename, year):
    if filename.startswith('서울특별시 공공자전거 대여이력 정보_{}년'.format(year)):
        # 서울시 정보소통광장에서 다운로드 받은 데이터의 인코딩이 cp949이기 때문에 인코딩을 지정해 주어야 한다.
        df_temp = pd.read_csv(folder_path+filename, encoding='cp949', low_memory=False)
        # 데이터에 불필요한 따옴표(')가 들어가 있다. 제거해 준다.
        df_temp = preprocessing(df_temp)
        # 2017년 중간에 컬럼명이 변경되었다. 2018년 이전 데이터에 대해서 최근 컬럼으로 컬럼명을 변경해 준다. 
        # 최근 컬럼명 cols는 위 셀에서 구한 것을 사용한다.
        if year < 2018 :
            df_temp.columns = cols
    elif filename.startswith('서울특별시 공공자전거 대여정보_{}'.format(year)):
        df_temp = pd.read_excel(folder_path+filename, encoding='cp949')
        df_temp.columns = cols
    else:
        df_temp = pd.DataFrame([])
    return  df_temp

In [9]:
%time df_2016 = concat_data(2016)
print(df_2016.shape)

서울특별시 공공자전거 대여이력 정보_2016년_2.csv (815301, 11)
서울특별시 공공자전거 대여이력 정보_2016년_3.csv (282189, 11)
서울특별시 공공자전거 대여이력 정보_2016년_1.csv (482990, 11)
(1580480, 11)
CPU times: user 9.04 s, sys: 1.02 s, total: 10.1 s
Wall time: 10.3 s
(1580480, 11)


In [10]:
%time df_2017 = concat_data(2017)
print(df_2017.shape)

서울특별시 공공자전거 대여이력 정보_2017년_2분기_2.csv (532503, 11)
서울특별시 공공자전거 대여이력 정보_2017년_2분기_1.csv (764154, 11)
서울특별시 공공자전거 대여이력 정보_2017년_3분기_2.csv (880529, 11)
서울특별시 공공자전거 대여이력 정보_2017년_3분기_1.csv (918911, 11)
서울특별시 공공자전거 대여이력 정보_2017년_4분기_1.csv (852503, 11)
서울특별시 공공자전거 대여이력 정보_2017년_4분기_2.csv (672623, 11)
서울특별시 공공자전거 대여이력 정보_2017년_1.csv (354731, 11)
(4975954, 11)
CPU times: user 43.5 s, sys: 5.53 s, total: 49 s
Wall time: 49.8 s
(4975954, 11)


In [11]:
%time df_2018 = concat_data(2018)
print(df_2018.shape)

서울특별시 공공자전거 대여이력 정보_2018년_1분기1.csv (784734, 11)
서울특별시 공공자전거 대여정보_201810_02.xlsx (366465, 11)
서울특별시 공공자전거 대여이력 정보_2018년_2분기_1.csv (677878, 11)
서울특별시 공공자전거 대여이력 정보_2018년_2분기_3.csv (566859, 11)
서울특별시 공공자전거 대여이력 정보_2018년_2분기_2.csv (951446, 11)
서울특별시 공공자전거 대여정보_201809_2.xlsx (385067, 11)
서울특별시 공공자전거 대여이력 정보_2018년_2분기_4.csv (586304, 11)
서울특별시 공공자전거 대여정보_201807_01.xlsx (1048575, 11)
서울특별시 공공자전거 대여정보_201811.xlsx (958253, 11)
서울특별시 공공자전거 대여정보_201808.xlsx (1026093, 11)
서울특별시 공공자전거 대여정보_201807_02.xlsx (41307, 11)
서울특별시 공공자전거 대여정보_201809_1.xlsx (1048575, 11)
(8441556, 14)
CPU times: user 10min 47s, sys: 15.8 s, total: 11min 3s
Wall time: 11min 27s
(8441556, 14)


In [12]:
df_2016.head()

,자전거번호,대여일시,대여대여소번호,대여대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M)
0,SPB-04076,2016-08-01 0:00,215,여의도고교 앞,3,2016-08-01 0:16,150,서강대역 2번출구 앞,4,15,4240
1,SPB-01179,2016-08-01 0:00,549,아차산역 3번출구,1,2016-08-01 0:06,573,구의문주차장 앞,10,5,930
2,SPB-03914,2016-08-01 0:00,141,연대 대운동장 옆,20,2016-08-01 0:10,107,신한은행 서교동금융센터점 앞,4,9,2080
3,SPB-03209,2016-08-01 0:01,510,서울숲 남문 버스정류소 옆,4,2016-08-01 0:51,510,서울숲 남문 버스정류소 옆,4,50,4090
4,SPB-03194,2016-08-01 0:01,510,서울숲 남문 버스정류소 옆,1,2016-08-01 0:49,510,서울숲 남문 버스정류소 옆,1,47,4070


In [13]:
df_2017.head()

,자전거번호,대여일시,대여대여소번호,대여대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M)
0,SPB-03589,2017-06-01 00:00:15,244,영등포삼환아파트 앞,5,2017-06-01 00:21:52,252,보라매역4번출구,2,21,5150
1,SPB-03229,2017-06-01 00:00:18,700,KB국민은행 염창역 지점 앞,17,2017-06-01 00:35:12,1123,공암나루근린공원 관리사무소옆,2,34,4820
2,SPB-05391,2017-06-01 00:00:21,1606,태릉입구역 3번출구,1,2017-06-01 00:28:20,1623,노원 구민회관,16,27,3170
3,SPB-04604,2017-06-01 00:00:29,1606,태릉입구역 3번출구,3,2017-06-01 00:28:45,1623,노원 구민회관,13,28,3170
4,SPB-00816,2017-06-01 00:00:29,419,홈플러스 앞,1,2017-06-01 00:35:54,926,불광역 8번출구,8,35,6240


In [14]:
df_2018.head()

,대여거치대,대여대여소명,대여대여소번호,대여소번호,대여일시,반납거치대,반납대여소명,반납대여소번호,반납일시,이용거리,이용거리(M),이용시간,이용시간(분),자전거번호
0,9,양평2나들목 보행통로 입구,227,NaN,2018-01-01 00:00:10,9,양평2나들목 보행통로 입구,227,2018-01-01 00:04:03,NaN,50.0,NaN,3.0,SPB-02041
1,15,"고속터미널역 8-1번, 8-2번 출구 사이",2219,NaN,2018-01-01 00:00:42,9,응봉역 1번출구,3511,2018-01-01 00:25:41,NaN,7670.0,NaN,24.0,SPB-03796
2,13,성수역 2번출구 앞,567,NaN,2018-01-01 00:00:58,2,영동대교 북단,3506,2018-01-01 00:09:33,NaN,1390.0,NaN,8.0,SPB-02402
3,5,청계8가사거리 부근,568,NaN,2018-01-01 00:01:02,9,왕십리역 11번 출구 앞,521,2018-01-01 00:10:13,NaN,1820.0,NaN,8.0,SPB-01564
4,4,동묘앞역 1번출구 뒤,361,NaN,2018-01-01 00:01:03,8,신당역 12번 출구 뒤,383,2018-01-01 00:06:18,NaN,850.0,NaN,4.0,SPB-05713


In [15]:
years = [2016, 2017, 2018]
for year in years:
    df = eval('df_'+str(year))
    df.to_csv('data/seoul-bike-{}.csv'.format(year), index=False)   
    print(df.shape)

(1580480, 11)
(4975954, 11)
(8441556, 14)


In [16]:
# 파일이 제대로 생성되었는지 확인한다.
%ls data/seoul-bike*

data/seoul-bike-2016.csv  data/seoul-bike-2017.csv  data/seoul-bike-2018.csv


In [17]:
# 생성된 파일 안에 내용이 제대로 들어가 있는지 확인한다.
df_2018 = pd.read_csv('data/seoul-bike-2018.csv', low_memory=False)
print(df_2018.shape)
df_2018.head()

(8441556, 14)


,대여거치대,대여대여소명,대여대여소번호,대여소번호,대여일시,반납거치대,반납대여소명,반납대여소번호,반납일시,이용거리,이용거리(M),이용시간,이용시간(분),자전거번호
0,9,양평2나들목 보행통로 입구,227,NaN,2018-01-01 00:00:10,9,양평2나들목 보행통로 입구,227,2018-01-01 00:04:03,NaN,50.0,NaN,3.0,SPB-02041
1,15,"고속터미널역 8-1번, 8-2번 출구 사이",2219,NaN,2018-01-01 00:00:42,9,응봉역 1번출구,3511,2018-01-01 00:25:41,NaN,7670.0,NaN,24.0,SPB-03796
2,13,성수역 2번출구 앞,567,NaN,2018-01-01 00:00:58,2,영동대교 북단,3506,2018-01-01 00:09:33,NaN,1390.0,NaN,8.0,SPB-02402
3,5,청계8가사거리 부근,568,NaN,2018-01-01 00:01:02,9,왕십리역 11번 출구 앞,521,2018-01-01 00:10:13,NaN,1820.0,NaN,8.0,SPB-01564
4,4,동묘앞역 1번출구 뒤,361,NaN,2018-01-01 00:01:03,8,신당역 12번 출구 뒤,383,2018-01-01 00:06:18,NaN,850.0,NaN,4.0,SPB-05713
